# Load model

In [3]:
import os
#os.environ["OPENAI_API_KEY"] = ""

In [4]:
from openai import OpenAI 
import os

MODEL="gpt-4o"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Load data

In [5]:
from datasets import load_dataset
import os

data_path = "../data/elaborations"

data_files_c2spo = {
    'train': os.path.join(data_path,"train","train_ds_c2spo.csv"),      
    'validation': os.path.join(data_path,"validation","valid_ds_c2spo.csv"),  
    'test': os.path.join(data_path,"test","test_ds_c2spo.csv")         
}

data_files_c2sp = {
    'train': os.path.join(data_path,"train","train_ds_c2sp.csv"),      
    'validation': os.path.join(data_path,"validation","valid_ds_c2sp.csv"),  
    'test': os.path.join(data_path,"test","test_ds_c2sp.csv")         
}

data_files_c2s = {
    'train': os.path.join(data_path,"train","train_ds_c2s.csv"),      
    'validation': os.path.join(data_path,"validation","valid_ds_c2s.csv"),  
    'test': os.path.join(data_path,"test","test_ds_c2s.csv")         
}

data_files_c4s = {
    'train': os.path.join(data_path,"train","train_ds_c4s.csv"),      
    'validation': os.path.join(data_path,"validation","valid_ds_c4s.csv"),  
    'test': os.path.join(data_path,"test","test_ds_c4s.csv")         
}

data_files_c4sp = {
    'train': os.path.join(data_path,"train","train_ds_c4sp.csv"),      
    'validation': os.path.join(data_path,"validation","valid_ds_c4sp.csv"),  
    'test': os.path.join(data_path,"test","test_ds_c4sp.csv")         
}

dataset = load_dataset('csv', data_files=data_files_c2spo)
setting = "c2spo"
print(setting)

c2spo


### Create dataframes

In [19]:
import pandas as pd
import os

df_train_spo = pd.DataFrame({
    "source_text": dataset["train"]["source_text"],
    "elaboration_sentence": dataset["train"]["elaboration_sentence"]
})

df_valid_spo = pd.DataFrame({
    "source_text": dataset["validation"]["source_text"],
    "elaboration_sentence": dataset["validation"]["elaboration_sentence"]
})

df_test_spo = pd.DataFrame({
    "source_text": dataset["test"]["source_text"],
    "elaboration_sentence": dataset["test"]["elaboration_sentence"]
})

df_train_spo.to_csv(os.path.join(data_path, "train_ds_c2spo_subjects_targets.csv"), index=False)
df_valid_spo.to_csv(os.path.join(data_path, "validation_ds_c2spo_subjects_targets.csv"), index=False)
df_test_spo.to_csv(os.path.join(data_path, "test_ds_c2spo_subjects_targets.csv"), index=False)

### Add columns

In [5]:
import pandas as pd
import os

df_train_s = pd.read_csv(os.path.join(data_path, "train_ds_s_subjects_targets.csv"))
df_train_sp = pd.read_csv(os.path.join(data_path, "train_ds_sp_subjects_targets.csv"))
df_train_spo = pd.read_csv(os.path.join(data_path, "train_ds_c2spo_subjects_targets.csv"))


df_valid_s = pd.read_csv(os.path.join(data_path, "validation_ds_s_subjects_targets.csv"))
df_valid_sp = pd.read_csv(os.path.join(data_path, "validation_ds_sp_subjects_targets.csv"))
df_valid_spo = pd.read_csv(os.path.join(data_path, "validation_ds_c2spo_subjects_targets.csv"))

df_test_s = pd.read_csv(os.path.join(data_path, "test_ds_s_subjects_targets.csv"))
df_test_sp = pd.read_csv(os.path.join(data_path, "test_ds_sp_subjects_targets.csv"))
df_test_spo = pd.read_csv(os.path.join(data_path, "test_ds_c2spo_subjects_targets.csv"))

df_train = df_train_spo
df_valid = df_valid_spo
df_test = df_test_spo

col_name = "target_sentence_4o"

dataset["train"] = dataset["train"].add_column(col_name, df_train[col_name])
dataset["validation"] = dataset["validation"].add_column(col_name, df_valid[col_name])
dataset["test"] = dataset["test"].add_column(col_name, df_test[col_name])

# Prompt and messages

In [6]:
SYSTEM_PROMPT_SUBJECT = """You are an expert in identifying the subject of the provided explanation sentence based on the context text. 
If the subject of the explanation sentence is a **pronoun (e.g., "it," "they," "he," "she")**, determine what the pronoun refers to within the context.
The subject MUST be written as a concise phrase (not as a complete sentence), and it MUST be found in the context text (it does not need to appear in the explanation sentence itself).
"""

SYSTEM_PROMPT_TARGET = """You are an expert in identifying the target phrase that provided explanation sentence is clarifying. 
Return the main phrase which explanation sentence is referring to.
The phrase must appear in the context text, but does not need to be present in the explanation sentence itself.
"""

SYSTEM_PROMPT_TARGET_SENT = """
You are an expert in identifying the sentence that the provided explanation sentence clarifies or refers to. 
Your task is to return the sentence from the CONTEXT TEXT, not the explanation sentence itself!
"""

SYSTEM_PROMPT_TARGET_SENT_AND_TARGET_PHRASE = """
You are an expert in identifying unclear or complex terms and concepts in a given text.
Your task is to:
1. Identify the sentence from the CONTEXT TEXT that the provided explanation sentence clarifies or refers to.
2. Specify the exact phrase within that sentence that is being clarified.
Return the identified sentence and the phrase from the CONTEXT TEXT, not the explanation sentence itself.
"""

SYSTEM_PROMPT_ELAB = """You are an expert in clarifying unclear or complex terms and concepts in a given text. Your task is as follows:  
1. Identify ***the target sentence** that requires clarification.  
2. Specify **the exact phrase** within that sentence that needs clarification. 
3. Determine the **type of clarification** that should be added. Choose from the following categories:  
- **Definition**: Provide a clear and concise definition of the term or concept.  
- **Example**: Offer one or more examples that illustrate the meaning of the term or concept.  
- **Background**: Provide relevant background knowledge or general statements to give context.  
- **Flow**: Describe the sequence or flow of actions to make the events clearer and easier to follow.  
- **Reason**: Explain the reason behind an action or occurrence.  
- **Result**: Clarify the outcome or result of an action or occurrence.  
- **Speculation**: Offer a possible outcome, assumption, or hypothesis, such as "what would happen if..." scenarios.  
- **Other**: If the clarification does not fit into the above categories, specify what kind of clarification you are providing. 
4. Based on the above's, generate **ONE concise explanation sentence** (made up of 10 words or fewer) that provides additional information related to the identified target sentence and phrase.  
The explanation sentence should be clear, plain, and simple in tone!"""

#Return the sentence from the context text that the explanation sentence is clarifying.

# Turn examples into ChatML format

In [7]:
def format_subject_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_SUBJECT},
            {"role": "user", "content":  "Identify the subject of the following explanation sentence: '{}' within the given text: '{}'".format(
    example["elaboration_sentence"], example["source_text"])}, #example["label_text"])},
        ]
    }

def format_target_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_TARGET},
            {"role": "user", "content":  "Identify the target of the following explanation sentence: '{}' within the given text: '{}'".format(
    example["elaboration_sentence"], example["label_text"])},
        ]
    }

def format_target_sent_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_TARGET_SENT},
            {"role": "user", "content":  "Identify the target sentence that the following explanation: '{}' clarifies or refers to within the given text: '{}'".format(
    example["elaboration_sentence"], example["source_text"])},
        ]
    }

def format_target_sent_and_target_phrase_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_TARGET_SENT_AND_TARGET_PHRASE},
            {"role": "user", "content":  "Identify the target sentence and target phrase in the context text: '{}' that is being clarified by the explanation sentence: '{}'".format(
    example["source_text"], example["elaboration_sentence"])},
        ]
    }

# for complex input text
def format_target_sent_complex_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_TARGET_SENT},
            {"role": "user", "content":  "Identify the target sentence that the following explanation: '{}' is clarifying within the given text: '{}'".format(
    example["elaboration_sentence"], example["source_text"])},
        ]
    }

def format_elab_example(example):
    return {
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT_ELAB},
            {"role": "user", "content":  "Identify the sentence and specific phrase in the given text: '{}' that need clarification. Then, specify the type of clarificatio needed and generate one concise explanation sentence.".format(
    example["source_text"])},
        ]
    }
    
formatted_train_dataset = dataset["train"].map(format_target_sent_and_target_phrase_example)
formatted_validation_dataset = dataset["validation"].map(format_target_sent_and_target_phrase_example)
formatted_test_dataset = dataset["test"].map(format_target_sent_and_target_phrase_example)

datasets = {"train":formatted_train_dataset, "validation":formatted_validation_dataset, "test":formatted_test_dataset}

In [17]:
formatted_test_dataset[0]["messages"][1]

{'content': "Identify the target sentence and target phrase in the context text: 'New companies have come that need skilled workers with more education. New Haven youth want those jobs, but they do not have the education or the skills.' that is being clarified by the explanation sentence: 'Many do not have the money to get the training they need.'",
 'role': 'user'}

# Identify subjects of elaborations

Analyze the subjects to explore the types of elaboration sentences.

In [16]:
import re

def extract_target_sentence(response: str) -> str:
    """
    Extract the target_sentence from the model's response.
    """
    match = re.search(r"target_sentence='(.*?)'", response)
    if match:
        return match.group(1) 
    else:
        raise ValueError("target_sentence not found.")

## Examples

### Subject

In [33]:
import random 
example = random.choice(formatted_test_dataset)
print(example["messages"][1],end="\n\n")

completion = client.beta.chat.completions.parse(
  model=MODEL,
  messages=example["messages"],
    response_format= ExplanationTarget,
)

response = completion.choices[0].message.parsed
print(response)

{'content': "Identify the subject of the following explanation sentence: 'It is not easy to relax, or to feel positive.' within the given text: 'In Philadelphia, there's a 50 percent higher prevalence of smoking among the poor than among the non-poor, according to Giridhar Mallya, director of policy and planning for the Philadelphia Department of Public Health. The morality of buying cigarettes when you're poor is complicated. The poor smoke to manage high levels of stress and depression, Mallya said, as much a part of poverty as empty pockets. It's also harder for the poor to get smoking-cessation counseling and nicotine patches than others who may receive help through insurance, experts said.'", 'role': 'user'}

subject='the poor'


### Target phrase

In [19]:
example = "Identify the target phrase in the following text: '{}' that is being clarified by the explanation sentence: '{}'".format(
    dataset["test"][100]["source_text"], dataset["test"][100]["elaboration_sentence"])

completion = client.beta.chat.completions.parse(
  model=MODEL,
  messages=[
    {"role": "system", "content": SYSTEM_PROMPT_TARGET_SENT_TARGET}, 
    {"role": "user", "content": example }  
  ],
    response_format= ExplanationTarget,
)

response = completion.choices[0].message.parsed
print(response)

subject='drones'


### Target sentence

In [14]:
import random 
from pydantic import BaseModel

class ExplanationTarget(BaseModel):
    target_sentence: str
    target_phrase: str
    
example = random.choice(formatted_test_dataset)
print(example["messages"][1],end="\n\n")

completion = client.beta.chat.completions.parse(
  model=MODEL,
  messages=example["messages"],
    response_format= ExplanationTarget,
)

response = completion.choices[0].message.parsed
print(response)

{'content': "Identify the target sentence and target phrase in the context text: 'He insisted that Iran's nuclear program is something it will not surrender. Analysts say Obama must be careful not to look too eager to make a deal.' that is being clarified by the explanation sentence: 'That would hurt his ability to bargain.'", 'role': 'user'}

target_sentence='Analysts say Obama must be careful not to look too eager to make a deal.' target_phrase='not to look too eager to make a deal'


### Elaboration

In [19]:
import random

seed = 42
sample_size = 50 
random_sample_dataset = formatted_test_dataset.shuffle(seed=seed).select(range(sample_size))

In [18]:
random_sample_dataset

Dataset({
    features: ['doc_num', 'source_text', 'label_text', 'elaboration_sentence', 'contextual_specificity_rating', 'messages'],
    num_rows: 50
})

In [13]:
import random 
from pydantic import BaseModel

example = random.choice(random_sample_dataset)
print(example["messages"][1],end="\n\n")


class ExplanationTarget(BaseModel):
    target_sentence: str
    target_phrase: str
    clarification_type: str
    explanation_sentence: str

completion = client.beta.chat.completions.parse(
  model=MODEL,
  messages=example["messages"],
    response_format= ExplanationTarget,
)

response = completion.choices[0].message.parsed
print(response)

{'content': "Identify the sentence and specific phrase in the given text: 'He said hate toward Roma people remains widespread there. His people still suffer unfair and insulting treatment, he said. They are also the first to be fired. He said that when poor Romani villagers come to Berlin, they find only overcrowded apartments.' that need clarification. Then, specify the type of clarificatio needed and generate one concise explanation sentence.", 'role': 'user'}

target_sentence='His people still suffer unfair and insulting treatment, he said.' target_phrase='unfair and insulting treatment' clarification_type='Example' explanation_sentence='Discrimination in housing and employment are common examples.'


In [40]:
df_results = pd.DataFrame({
    "doc_num": formatted_test_dataset["doc_num"],
    "source_text" : formatted_test_dataset["source_text"],
    "label_text" : formatted_test_dataset["label_text"],
    "elaboration_sentence":formatted_test_dataset["elaboration_sentence"],
    "response":"",
    "pred_elaboration":"",
    "target_sentence_target": "",
    "target_sentence_4o":"",
    "clarification_category":"",
    
})

df_results.to_csv(f"../data/elaborations/{split}_ds_c2spo_gpt4o_elab_preds_targets.csv", index=False)

## Create elaboration targets df

In [11]:
import pandas as pd
splits = ["train","validation","test"]

for split in splits:
    df_results = pd.DataFrame({
        "doc_num": datasets[split]["doc_num"],
        "source_text" : datasets[split]["source_text"],
        "label_text" : datasets[split]["label_text"],
        "elaboration_sentence":datasets[split]["elaboration_sentence"],
        "response":"",
        "target_sentence_4o":"",
        "target_sentence_target":"",
    })
    df_results.to_csv(f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv", index=False)
print(setting)

c2spo


## Get subjects, targets and elaboration info

In [20]:
import pandas as pd
from tqdm.notebook import tqdm

splits = ["train","validation","test"]
split = "train"

df_results = pd.read_csv(f"../data/elaborations/{split}_ds_c2spo_subjects_targets.csv")

formatted_dataset = formatted_train_dataset
column_name = "target-phrase"
df_results[column_name] = "" #df_results[column_name].fillna("")

for idx, example in tqdm(enumerate(formatted_dataset),total=len(formatted_dataset)):
    if df_results.at[idx,column_name]=="":
        completion = client.chat.completions.create(model=MODEL,messages=example["messages"])
        df_results.at[idx,column_name] = completion.choices[0].message.content

  0%|          | 0/1049 [00:00<?, ?it/s]

In [24]:
df_results.rename(columns={"target-phrase":"elaboration-info"},inplace=True)

In [28]:
df_results.to_csv(f"../data/elaborations/{split}_ds_c2spo_subjects_targets.csv", index=False)

## Get target sentences for elaborations in specified response format

In [23]:
from pydantic import BaseModel

# for elaboration targets identification
class ExplanationTarget(BaseModel):
    #explanation_sentence: str
    subject: str
    #target_sentence: str
    #target_phrase: str

# for elaboration generation
class ExplanationTarget(BaseModel):
    target_sentence: str
    target_phrase: str
    #clarification_type: str
    #explanation_sentence: str

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm
from pydantic import BaseModel

class ExplanationTarget(BaseModel):
    target_sentence: str
    target_phrase: str

splits = ["train","validation","test"]

for split in splits:
    df_results = pd.read_csv(f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv")
    #df_results = pd.read_csv(f"../data/elaborations/{split}_ds_c2spo_gpt4o_elab_preds_targets.csv")
    column_name = "response" # column_names : "subject " "target_sentence_4o", "target_sentence_target"
    df_results[column_name] = df_results[column_name].fillna("")
    formatted_dataset = datasets[split]
    
    for idx, example in tqdm(enumerate(formatted_dataset),total=len(formatted_dataset)):
        if df_results.at[idx,column_name] == "":
            try:
                completion = client.beta.chat.completions.parse(model=MODEL,messages=example["messages"],response_format=ExplanationTarget)
                df_results.at[idx,column_name] = completion.choices[0].message.parsed
                df_results.to_csv(f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv",index=False)
            except Exception as e:
                print(f"{e} for index {idx} in {split}")
                df_results.at[idx, column_name] = "filtered"

# Save results

In [52]:
len(df_results[df_results[column_name]==""])

0

In [55]:
df_results.to_csv("../data/elaborations/test_ds_sp_subjects_targets.csv", index=False)

## Create additional df for prior-context only datasets

In [56]:
import pandas as pd
from tqdm.notebook import tqdm

split = "test"
dataset = load_dataset('csv', data_files=data_files_c2s)
df_sp = pd.read_csv(f"../data/elaborations/{split}_ds_sp_subjects_targets.csv")
df_sp = df_sp.drop_duplicates(subset="elaboration_sentence", keep="first")
df = pd.read_csv(f"../data/elaborations/{split}_ds_s_subjects_targets.csv")
"""df = pd.DataFrame({
    "source_text" : dataset[split]["source_text"],
    "label_text" : dataset[split]["label_text"],
    "elaboration_sentence":dataset[split]["elaboration_sentence"],
    
}) """

'df = pd.DataFrame({\n    "source_text" : dataset[split]["source_text"],\n    "label_text" : dataset[split]["label_text"],\n    "elaboration_sentence":dataset[split]["elaboration_sentence"],\n    \n}) '

In [57]:
print(len(df_sp))
print(len(df))

116
116


In [ ]:
# merge the additional columns from df_sp based on elaboration_sentence
columns_to_add = ["target_sentence_target"]
#columns_to_add = ["subject", "target", "elaboration_info", "target_sentence_4o"]
df = pd.merge(df, df_sp[["elaboration_sentence"] + columns_to_add], on="elaboration_sentence", how="left")
df

In [60]:
df.to_csv(f"../data/elaborations/{split}_ds_s_subjects_targets.csv")

In [59]:
idx= 100
print(df.loc[idx,"target_sentence_4o"], end="\n\n")
print(df.loc[idx, column_name])

target_sentence='Drones are small flying aircraft.'

target_phrase='Drones'


# Check the targets

In [10]:
import pandas as pd

split = "validation"
setting = "c4spo"
filename = f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv" #f"../data/elaborations/{split}_ds_{setting}_subjects_targets.csv"#f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv"
df_t = pd.read_csv(filename)
print(filename)

../data/elaborations/validation_ds_c4spo_elab_targets.csv


### Extract responses

In [4]:
import re

def extract_target_components(response):

    target_sentence = None
    target_phrase = None

    target_sentence, target_phrase = response.split("target_phrase=")
    target_sentence = target_sentence.split("target_sentence=")[1]

    return target_sentence, target_phrase

response = "target_sentence='He called the talk a \"fireside hangout.\"' target_phrase='\"fireside hangout\"'"
target_sentence, target_phrase = extract_target_components(response)
print(target_sentence)
print(target_phrase)

'He called the talk a "fireside hangout."' 
'"fireside hangout"'


In [11]:
df_t["target_sentence_4o"] = df_t["target_sentence_4o"].astype("str")
df_t["target_sentence_target"] = df_t["target_sentence_target"].astype("str")

for idx, row in df_t.iterrows():
    response = row["response"]  
    if response:
        try: 
            target_sentence, target_phrase = extract_target_components(response)
            df_t.at[idx, "target_sentence_4o"] = target_sentence
            df_t.at[idx, "target_sentence_target"] = target_phrase
        except Exception as e:
            df_t.at[idx, "target_sentence_4o"] = ""
            df_t.at[idx, "target_sentence_target"] = ""

df_t.isnull().sum()

doc_num                     0
source_text                 0
label_text                134
elaboration_sentence        0
response                    0
target_sentence_4o          0
target_sentence_target      0
dtype: int64

## Check for duplicates (target sentence vs elaboration sentence)

In [12]:
from tqdm.notebook import tqdm
from bert_score import BERTScorer

scorer = BERTScorer(model_type='bert-base-uncased')#,device='cuda:0')

bert_scores_f1 = []

for index, row in tqdm(df_t.iterrows(), total=len(df_t)):
    elaboration = row['elaboration_sentence']
    target = row['target_sentence_4o']
    
    #  BERTScore for this pair
    P, R, F1 = scorer.score(
        cands=[target],  
        refs=[elaboration],              
    )
    
    bert_scores_f1.append(F1.mean().item())

df_t["target_sentence_4o_bsf1"] = bert_scores_f1

  0%|          | 0/134 [00:00<?, ?it/s]

In [9]:
df_t.to_csv(filename,index=False)
print(filename)

../data/elaborations/train_ds_c4spo_elab_targets.csv


In [15]:
df_check = df_t[df_t["target_sentence_4o_bsf1"]>0.4]
for index, row in df_check.iterrows():
    print("ID:", index)
    print("Similarity score:", round(row["target_sentence_4o_bsf1"],3))
    print(row["source_text"], end="\n\n")
    print("Elab:",row["elaboration_sentence"], end="\n\n")
    print("Target sent:",row["target_sentence_4o"], end="\n\n")
    print("-"*120)
    #df_t.at[index,"target_sentence_4o"]=""

ID: 6
Similarity score: 0.409
With a bright purple stuffed animal in tow, Emily Whitehead and her parents, Tom and Kari, of Philipsburg, Pa., walked the hallways of the U.S. Capitol on Thursday, urging legislators to support the research that saved Emily's life. Grollman said that unlike other hospitals around the country, children's hospitals didn't receive funding for residency training until the early 1990s when the Children's Hospitals Graduate Medical Education program was created. Emily recently celebrated one year of being cancer-free after a recurring battle with acute lymphoblastic leukemia, or ALL, the most common form of pediatric cancer.

Elab: In the spring of 2012, Emily was close to death.

Target sent: 'With a bright purple stuffed animal in tow, Emily Whitehead and her parents, Tom and Kari, of Philipsburg, Pa., walked the hallways of the U.S. Capitol on Thursday, urging legislators to support the research that saved Emily's life. Grollman said that unlike other hospit

In [8]:
idx = 665
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = ""

In [21]:
idx = 38
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = ""

In [22]:
idx = 65
df_t.at[idx,"target_sentence_4o"] = "'The letter warns that if something isn't done soon to boost the ranks of plant scientists, breeders, students, and others in the field, horticulture could become a lost art and a forgotten science.'"
df_t.at[idx,"target_sentence_target"] = "'the letter'"

In [23]:
idx = 87
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = "rock snot"

In [24]:
idx = 108
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = "'every ill patient'"

In [25]:
idx = 110
df_t.at[idx,"target_sentence_4o"] = "'Rapamycin also seems to have an anti-inflammatory effect.'"
df_t.at[idx,"target_sentence_target"] = "'Rapamycin'"

In [9]:
idx = 563
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = ""

In [10]:
idx = 574
df_t.at[idx,"target_sentence_4o"] = "'The government says, ‘Pay your taxes in artwork.'"
df_t.at[idx,"target_sentence_target"] = "'The government'"

In [11]:
idx = 692
df_t.at[idx,"target_sentence_4o"] = "'Clay Barker, the Kansas Republican Party executive director, said this shouldn't be an issue for voters. '"
df_t.at[idx,"target_sentence_target"] = "'this shouldn't be an issue for voters'"

In [12]:
idx = 693
df_t.at[idx,"target_sentence_4o"] = "'Clay Barker, the Kansas Republican Party executive director, said this shouldn't be an issue for voters. '"
df_t.at[idx,"target_sentence_target"] = "'voters'"

In [13]:
idx = 717
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = ""

In [14]:
idx = 787
df_t.at[idx,"target_sentence_4o"] = "'Beginning in 1952 and continuing sporadically over the next 11 years, John Adams Jr. dug up dozens of items that Union and Confederate soldiers had deposited during the October 1864 Battle of Westport.'"
df_t.at[idx,"target_sentence_target"] = "'John Adams Jr. dug up dozens of items'"

In [15]:
idx = 978
df_t.at[idx,"target_sentence_4o"] = ""
df_t.at[idx,"target_sentence_target"] = ""

In [52]:
df_targets.iloc[42]

Unnamed: 0                                                                42
source_text                Apple also agreed to change the way it bills c...
label_text                 Apple also agreed to change the way it bills c...
elaboration_sentence                          Tim Cook is the head of Apple.
subject                                                             Tim Cook
target                                                              Tim Cook
elaboration_info           The sentence from the context text that the ex...
target_sentence_4o          target_sentence='Tim Cook is the head of Apple.'
target_sentence_target                         target_phrase='head of Apple'
target_sentence_4o_bsf1                                             0.669486
messages                   [{'role': 'system', 'content': '
You are an ex...
Name: 42, dtype: object

In [ ]:
df_targets.at[42,"target_sentence_4o"] = "He said the company gave in to the FTC."
df_targets.at[47,"target_sentence_4o"] = "It wouldn't be surprising if other forms of popular online gaming were prohibited by the government too."

## Correct

In [53]:
import pandas as pd
from tqdm.notebook import tqdm
from pydantic import BaseModel


df_targets = pd.read_csv(f"../data/elaborations/test_ds_sp_subjects_targets.csv")
column_name = "target_sentence_4o" 
df_targets[column_name] = df_targets[column_name].fillna("")


class ExplanationTarget(BaseModel):
    #subject: str
    target_sentence: str

SYSTEM_PROMPT_TARGET_SENT = """
You are an expert in identifying the sentence from the CONTEXT TEXT that the provided explanation sentence clarifies or refers to. 
Your task is to return the sentence from the CONTEXT TEXT, not the explanation sentence itself.
"""

def format_target_sent_example(example):
    return [
            {"role": "system", "content": SYSTEM_PROMPT_TARGET_SENT},
            {"role": "user", "content":  "Identify the target sentence that the following explanation: '{}' clarifies or refers to within the given text: '{}'".format(
    example["elaboration_sentence"], example["label_text"])},
        ]

df_targets["messages"] = df_targets.apply(format_target_sent_example, axis=1)

for idx, example in tqdm(df_targets.iterrows(),total=len(df_targets)):
    if df_targets.at[idx,column_name] == "":
        completion = client.beta.chat.completions.parse(model=MODEL,messages=example["messages"],response_format=ExplanationTarget)
        df_targets.at[idx,column_name] = completion.choices[0].message.parsed

  0%|          | 0/116 [00:00<?, ?it/s]

In [48]:
print(df_targets["messages"].iloc[42])

[{'role': 'system', 'content': '\nYou are an expert in identifying the sentence from the CONTEXT TEXT that the provided explanation sentence clarifies or refers to. \nYour task is to return the sentence from the CONTEXT TEXT, not the explanation sentence itself.\n'}, {'role': 'user', 'content': 'Identify the target sentence that the following explanation: \'Tim Cook is the head of Apple.\' clarifies or refers to within the given text: \'Apple also agreed to change the way it bills customers. It will let people know when someone buys something on a game or app. Tim Cook is the head of Apple. He said the company gave in to the FTC. The agreement does not ask Apple "to do anything we weren\'t already going to do."\''}]


In [55]:
df_targets.loc[,"target_sentence_4o"]

ExplanationTarget(target_sentence='When the people who make the fantasy teams bet money that their players will score the most points, it is considered gambling.')

In [13]:
import pandas as pd

split = "train"
setting = "c4sp"
filename = f"../data/elaborations/train/{split}_ds_{setting}_elab_targets.csv" #f"../data/elaborations/{split}_ds_{setting}_subjects_targets.csv"#f"../data/elaborations/{split}_ds_{setting}_elab_targets.csv"
df_t = pd.read_csv(filename)

In [14]:
df_t[df_t["target_sentence_4o"]==""]

,doc_num,source_text,label_text,elaboration_sentence,response,target_sentence_4o,target_sentence_target
